In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler # 적절한 타이밍에 데이터 셰이프를 변환하거나, 데이터를 플롯하거나 표준화 가능

from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense
from keras.layers import Dropout, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping

# 데이터 불러오기

In [ ]:
train_x = pd.read_csv('./data/train_x_df.csv')
test_x = pd.read_csv('./data/test_x_df.csv')
train_y = pd.read_csv('./data/train_y_df.csv')

# 데이터 전처리

In [ ]:
# 중간값 구하는 함수
def getMidprice(sample_id, df):
  sample_id_df=df[df['sample_id']==sample_id]

  high_prices = sample_id_df.loc[:, 'high'].values
  low_prices = sample_id_df.loc[:, 'low'].values
  mid_prices = (high_prices + low_prices)/2.0
  
  mid_prices=mid_prices.reshape(-1, 1) # scaler.fit_transform

  return mid_prices

In [ ]:
# 훈련 및 검증 데이터셋 생성 함수
# look_back : 관측값에서 유지해야 할 가격의 수, 예측전 지난 n개의 가격을 돌아보도록 함
# foresight : 각 train 시퀀스에 대한 라벨이 시퀀스 다음 n+1분의 가격
# dataset변수의 길이를 조정해서 넣어서 train 및 validation 각각 생성
def create_train_dataset(dataset, look_back=210, foresight=119):
  X, Y = [], []

  for i in range(len(dataset)-look_back-foresight) :
    # 관찰 값을 형성하는 특징으로 과거 210개의 가격 시퀀스 지정
    obs=dataset[i:(i+look_back), 0]
    # 시퀀스 추가
    X.append(obs)
    # 210개 가격의 한 시퀀스의 120분 후 가격 
    Y.append(dataset[i+(look_back+foresight), 0])

  return np.array(X), np.array(Y)

In [ ]:
# 테스트 데이터셋 생성 함수
def create_test_dataset(dataset, look_back=210):
  x_test = []

  for i in range(len(dataset)-look_back) :
    # 관찰 값을 형성하는 특징으로 과거 210개의 가격 시퀀스 지정
    obs=dataset[i:(i+look_back), 0]
    # 시퀀스 추가
    x_test.append(obs)

  return np.array(x_test)

# 학습 및 예측
- 대회에서 제공한 train dataset을 사용하지 않고 test 데이터만을 통해 예측했습니다.
- 최대 예측값이 1보다 작으면 sell_time을 0으로 설정 했습니다.

In [ ]:
samples=test_x['sample_id'].unique()
pred_selling_time = []

for idx, id in tqdm(enumerate(samples)):
  print(idx, "/ sample_id :", id)
  print("get mid price")
  # sample_id에 대한 중간값으로 학습 및 테스트 데이터 생성
  x_mid_prices = getMidprice(id, test_x)

  ##############################################################################

  print("windowed normalization")
  ### 윈도우 방식 표준화
  # 데이터를 표준화하기 위한 윈도우 크기
  normalization_window = 138

  # 표준화 범위
  scaler = MinMaxScaler(feature_range=(0, 1))

  x_win_prices=x_mid_prices.copy()
  # 한 번에 138개의 x_train 데이터별로 윈도우 표준화
  for i in range(0, 1380, normalization_window):
    # 현재 윈도우에 대해 스케일러 객체를 데이터에 적용
    scaler.fit(x_mid_prices[i:i+normalization_window, :])
    # 현재 윈도우의 데이터를 선택한 특징 범위(0~1)의 데이터로 변환
    x_win_prices[i:i+normalization_window,:] = scaler.transform(x_mid_prices[i:i+normalization_window,:])

  ##############################################################################

  print("Exponential smoothing")
  # 지수 평활법 - 최근 사건이 과거보다 현재 데이터에 더 많은 영향을 주도록
  # 데이터에 포함된 고주파 노이즈 제거

  Smoothing=0 # 평활화 값을 0으로 초기화
  gamma = 0.3 # 소멸 계수
  x_sm_prices=[]

  # x_train데이터 평활화
  for a in range(1380):
    # 평활화 값 업데이트
    Smoothing = gamma*x_win_prices[a]+(1-gamma)*Smoothing
    # 평활화 값으로 데이터 포인트 값을 대체
    x_sm_prices.append(Smoothing)

  x_sm_prices=np.array(x_sm_prices).reshape(-1, 1)

  ##############################################################################

  print("get train and validation data set")
  x_train, y_train = createTrainValid(x_sm_prices[:1259])
  x_validation, y_validation = createTrainValid(x_sm_prices)
  x_test = createTestset(x_sm_prices)
  
  ##############################################################################

  print("reshaping")
  x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
  y_train = np.reshape(y_train, (y_train.shape[0], 1, 1)) 

  x_validation = np.reshape(x_validation, (x_validation.shape[0], 1, x_validation.shape[1]))
  y_validation = np.reshape(y_validation, (y_validation.shape[0], 1, 1)) 

  x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

  ##############################################################################

  print("training")
  # seed 값 설정
  seed = 0
  np.random.seed(seed)
  tf.random.set_seed(3)

  model=Sequential()
  model.add(LSTM(480, input_shape=(1, 210), dropout=0.1, recurrent_dropout=0.2, return_sequences=True))
  model.add(LSTM(480, dropout=0.1, recurrent_dropout=0.2, return_sequences=True))
  model.add(LSTM(480, dropout=0.1, recurrent_dropout=0.2))
  model.add(Dense(1, activation='linear'))

  model.compile(loss='mae', optimizer='adam')

  early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

  # 모델의 실행
  history = model.fit(x_train, y_train, validation_data=(x_validation, y_validation), 
                      epochs=50, batch_size=3000, verbose=0, callbacks=[early_stopping_callback])

  preds = model.predict(x_test)[-120:]
  preds = preds.flatten()

  st = 0 
  if scaler.inverse_transform(preds.max().reshape(-1,1))/x_mid_prices[-1,0] > 1:
    st = preds.argmax()
  pred_selling_time.append(st)

pred_selling_time

In [ ]:
submission = pd.DataFrame()
submission['sample_id'] = range(x_train_open.shape[0], x_train_open.shape[0]+x_test_open.shape[0])
submission['buy_quantity'] = 1
submission['sell_time'] = pred_selling_time
submission

In [ ]:
# submission.to_csv('submission_lstm.csv', index=False)